# SQLCoder

Welcome to the third part of the tutorial series on building a chatbot over a corpus of private
documents using Large Language Models (LLMs).

In this Notebook, you set up an SQLCoder, an Large Language Model (LLM) that can take a user's
question and turn it into a SQL query that can be run against a database. This is a powerful tool
that can be used to query a database using natural language.

## Table of Contents

1. [Creating the SQLCoder Deployment](#creating-the-sqlcoder-deployment)
1. [Conclusion and Next Steps](#conclusion-and-next-steps)

In [ ]:
import subprocess

# Creating the SQLCoder Deployment

Apply the deployment spec below to create the SQLCoder deployment. This will create a new deployment
with the SQLCoder model and expose it within the cluster using a Kubernetes service. Make sure to
provide the PVC name that houses the SQLCode name.

In [ ]:
model_pvc = "..."  # PVC name here

sql_llm_deploy = """
apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    io.kompose.service: sql-llm
  name: sql-llm
spec:
  progressDeadlineSeconds: 600
  replicas: 1
  selector:
    matchLabels:
      io.kompose.service: sql-llm
  strategy:
    type: Recreate
  template:
    metadata:
      labels:
        hpe-ezua/app: ezaf-other
        hpe-ezua/type: infra-service
        io.kompose.network/nvidia-llm: "true"
        io.kompose.service: sql-llm
    spec:
      containers:
      - args:
        - --model
        - defog/sqlcoder-7b-2
        - --tensor-parallel-size
        - "2"
        env:
        - name: HF_HOME
          value: /cache
        - name: NUMBA_CACHE_DIR
          value: /tmp/numba_cache
        image: vllm/vllm-openai:v0.4.1
        imagePullPolicy: IfNotPresent
        name: sqlcoder-server
        ports:
        - containerPort: 8000
          hostPort: 8000
          name: 8000-tcp
          protocol: TCP
        resources:
          limits:
            nvidia.com/gpu: "2"
          requests:
            nvidia.com/gpu: "2"
        volumeMounts:
        - mountPath: /cache
          name: sql-llm-claim
      priorityClassName: sql-bot
      restartPolicy: Always
      volumes:
      - name: sql-llm-claim
        persistentVolumeClaim:
          claimName: {0}
---
apiVersion: v1
kind: Service
metadata:
  labels:
    io.kompose.service: sql-llm
  name: sql-llm
spec:
  ports:
  - name: 8000-tcp
    port: 8000
    protocol: TCP
    targetPort: 8000
  selector:
    io.kompose.service: sql-llm
  type: ClusterIP
""".format(model_pvc)

with open("sql-llm.yaml", "w") as f:
    f.write(sql_llm_deploy)

subprocess.run(["kubectl", "apply", "-f", "sql-llm.yaml"])

# Conclusion and Next Steps

Congratulations on completing this crucial step in this tutorial series! You've successfully
deployed an SQLCoder LLM that can take a user's question and turn it into a SQL query that can be
run against a database.